In [1]:
import requests
import json
import pandas as pd
import re
from pprint import pprint

# Import the API key
from config import ombdapi_key

In [2]:
# read oscar winning movie list
oscar_df = pd.DataFrame( pd.read_csv("../project1-team8/Resources/oscar.csv", index_col='Unnamed: 0') )

# remove blank film names
oscar_df = oscar_df.dropna()

In [3]:
# daily API limit 1000
oscar_df = oscar_df.iloc[0:700]

# rename oscar df to avoide overlap with imdb data
for name in oscar_df.columns :
    oscar_df = oscar_df.rename( columns={ name : f'Oscar_{name}' })
    
oscar_df = oscar_df.reset_index(drop=True) # when select partial for API
oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination
0,Everything Everywhere All at Once,2022,7,11
1,All Quiet on the Western Front,2022,4,9
2,The Whale,2022,2,3
3,Top Gun: Maverick,2022,1,6
4,Black Panther: Wakanda Forever,2022,1,5
...,...,...,...,...
695,The Sting,1973,7,10
696,The Exorcist,1973,2,10
697,The Way We Were,1973,2,6
698,Cries and Whispers,1973,1,5


In [4]:
# omdb api preparation
base_url = "http://www.omdbapi.com/?"
params = { "apikey" : ombdapi_key, "type" : "movie" }

# general info to extract from api. column names are key in api.
general_col = ['Year', 'Rated', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', \
               'imdbRating', 'imdbVotes','BoxOffice']

for i in range(len(general_col)) :
    oscar_df[ general_col[i] ] = None

# non omdb rating to fulfill
ratings_col = ['Ratings_IMD', 'Ratings_Rotten_Tomatoes' ,'Ratings_Metacritic']

for i in range(len(ratings_col)) :
    oscar_df[ ratings_col[i] ] = None


In [5]:
# request API

for index in range(len(oscar_df)) :
    
    # export movie by film name in oscar df
    params["t"] = re.sub('\s+', '+', oscar_df.loc[ index , 'Oscar_Film' ] ) 
    movie_data = requests.get( base_url , params ).json()
    
    # API has response == False when unable to find or excited limit
    if movie_data['Response'] == 'True' :        
        
        # extract general info and fulfill the dataframe, by colnames = keys
        for info_col in general_col :  

            try :
                oscar_df.loc[ index , info_col ] = movie_data[ info_col ]       
            except KeyError :
                oscar_df.loc[ index , info_col ] = None

        # extract other ratings and fill rating columns
        try :
            temp_rating = pd.DataFrame( movie_data[ 'Ratings' ] ).set_index("Source")
        except KeyError :
            print( f'Film: {params["t"]}. Missing Other Ratings')
            oscar_df.loc[ index , 'Ratings_IMD'] = None
            oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = None
            oscar_df.loc[ index , 'Ratings_Metacritic'] = None

        try :
            oscar_df.loc[ index , 'Ratings_IMD'] = temp_rating.loc[ "Internet Movie Database" ,'Value']
        except KeyError :
            oscar_df.loc[ index , 'Ratings_IMD'] = None

        try :
            oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = temp_rating.loc[ "Rotten Tomatoes" ,'Value']
        except KeyError :
            oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = None

        try :
            oscar_df.loc[ index , 'Ratings_Metacritic'] = temp_rating.loc["Metacritic" ,'Value']
        except KeyError :
            oscar_df.loc[ index , 'Ratings_Metacritic'] = None
    
    else :
        print( f'API Unable to match Film: {params["t"]}')


API Unable to match Film: Period.+End+of+Sentence.
API Unable to match Film: Flesh+and+Sand
API Unable to match Film: La+Maison+en+Petits+Cubes
API Unable to match Film: Peter+&amp;+the+Wolf
API Unable to match Film: Le+Mozart+des+pickpockets
API Unable to match Film: Hustle+&amp;+Flow
API Unable to match Film: Wallace+&amp;+Gromit:+The+Curse+of+the+Were-Rabbit
API Unable to match Film: Lemony+Snicket's+A+Series+of+Unfortunate+Events
Film: Adaptation. Missing Other Ratings
API Unable to match Film: Il+Postino:+The+Postman
API Unable to match Film: Thelma+&amp;+Louise
API Unable to match Film: Hôtel+Terminus:+The+Life+and+Times+of+Klaus+Barbie
API Unable to match Film: The+Ten-Year+Lunch
API Unable to match Film: Anna+&amp;+Bella
API Unable to match Film: Fanny+&amp;+Alexander
API Unable to match Film: Flamenco+at+5:15
API Unable to match Film: Just+Another+Missing+Kid
API Unable to match Film: To+Begin+Again
API Unable to match Film: Who+Are+the+DeBolts?+And+Where+Did+They+Get+Nineteen

In [6]:
oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination,Year,Rated,Runtime,Genre,Director,Writer,...,Plot,Language,Country,Awards,imdbRating,imdbVotes,BoxOffice,Ratings_IMD,Ratings_Rotten_Tomatoes,Ratings_Metacritic
0,Everything Everywhere All at Once,2022,7,11,2022,R,139 min,"Action, Adventure, Comedy","Daniel Kwan, Daniel Scheinert","Daniel Kwan, Daniel Scheinert",...,A middle-aged Chinese immigrant is swept up in...,"English, Mandarin, Cantonese",United States,Won 7 Oscars. 372 wins & 357 nominations total,7.8,"445,131","$77,191,785",7.8/10,93%,81/100
1,All Quiet on the Western Front,2022,4,9,2022,R,148 min,"Action, Drama, War",Edward Berger,"Edward Berger, Lesley Paterson, Ian Stokell",...,A young German soldier's terrifying experience...,"German, French","Germany, United States, United Kingdom",Won 4 Oscars. 48 wins & 81 nominations total,7.8,"205,354",N/A,7.8/10,90%,76/100
2,The Whale,2022,2,3,2022,R,117 min,Drama,Darren Aronofsky,Samuel D. Hunter,...,"A reclusive, morbidly obese English teacher at...",English,United States,Won 2 Oscars. 44 wins & 116 nominations total,7.7,"152,388","$17,463,630",7.7/10,64%,60/100
3,Top Gun: Maverick,2022,1,6,2022,PG-13,130 min,"Action, Drama",Joseph Kosinski,"Jim Cash, Jack Epps Jr., Peter Craig",...,"After thirty years, Maverick is still pushing ...",English,United States,Won 1 Oscar. 93 wins & 214 nominations total,8.3,"590,794","$718,732,821",8.3/10,96%,78/100
4,Black Panther: Wakanda Forever,2022,1,5,2022,PG-13,161 min,"Action, Adventure, Drama",Ryan Coogler,"Ryan Coogler, Joe Robert Cole, Stan Lee",...,The people of Wakanda fight to protect their h...,"English, Xhosa, Maya, Spanish, Haitian, French",United States,Won 1 Oscar. 44 wins & 154 nominations total,6.7,"270,203","$453,829,060",6.7/10,84%,67/100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,The Sting,1973,7,10,1973,PG,129 min,"Comedy, Crime, Drama",George Roy Hill,David S. Ward,...,Two grifters team up to pull off the ultimate ...,English,United States,Won 7 Oscars. 18 wins & 6 nominations total,8.3,"269,448","$156,000,000",8.3/10,92%,83/100
696,The Exorcist,1973,2,10,1973,R,122 min,Horror,William Friedkin,William Peter Blatty,...,When a teenage girl is possessed by a mysterio...,"English, Latin, Greek, French, German, Arabic,...",United States,Won 2 Oscars. 17 wins & 17 nominations total,8.1,"421,000","$233,005,644",8.1/10,84%,81/100
697,The Way We Were,1973,2,6,1973,PG,118 min,"Drama, Romance",Sydney Pollack,"Arthur Laurents, Francis Ford Coppola, Paddy C...",...,"During pre-WWII McCarthyism, a diametrically o...","English, French",United States,Won 2 Oscars. 6 wins & 8 nominations total,7.0,"26,279","$45,000,000",7.0/10,64%,61/100
698,Cries and Whispers,1973,1,5,2015,N/A,13 min,"Documentary, Short",Kogonada,N/A,...,N/A,English,United States,N/A,N/A,7,N/A,7.0/10,64%,61/100


In [7]:
oscar_df.to_csv('.\Resources\oscar_omdb_1.csv', index=False)